# ML-5333 Project 2 - MLB Data Collection

In [16]:
import pandas as pd
import numpy as np
import baseball_scraper
import pybaseball
import statsapi
import mlbgame

In [56]:
with open('data\\team_names.txt', 'r') as file:
    team_list = [x.replace('\n','') for x in file.readlines()]
team_df = []
for team in team_list:
    team_df.append(statsapi.lookup_team(team)[0])
team_df = pd.DataFrame.from_dict(team_df)
team_df.columns = ['statsapi_id','name','team_code','file_code','team_name','loc_name','short_name']

In [57]:
team_df.head()

,statsapi_id,name,team_code,file_code,team_name,loc_name,short_name
0,144,Atlanta Braves,atl,atl,Braves,Atlanta,Atlanta
1,146,Miami Marlins,mia,mia,Marlins,Miami,Miami
2,121,New York Mets,nyn,nym,Mets,Flushing,NY Mets
3,143,Philadelphia Phillies,phi,phi,Phillies,Philadelphia,Philadelphia
4,120,Washington Nationals,was,was,Nationals,Washington,Washington
